In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
# Define constants
CLASS_NAMES = ['CaS', 'CoS', 'Gum', 'MC', 'OC', 'OLP', 'OT']
IMG_SIZE = 224
BATCH_SIZE = 32

# Set up path
base_dir = '/kaggle/input/teeth-dataset/Teeth_Dataset'
train_dir = os.path.join(base_dir, 'Training')
valid_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Testing')

# Verify paths and data
print("Available directories:")
print(f"Training: {os.path.exists(train_dir)}")
print(f"Validation: {os.path.exists(valid_dir)}")
print(f"Testing: {os.path.exists(test_dir)}")

Available directories:
Training: True
Validation: True
Testing: True


In [4]:
# Count images in each directory
def count_images(directory):
    counts = {}
    for class_name in CLASS_NAMES:
        class_dir = os.path.join(directory, class_name)
        if os.path.exists(class_dir):
            counts[class_name] = len(os.listdir(class_dir))
    return counts

print("\nImage counts:")
print("Training:", count_images(train_dir))
print("Validation:", count_images(valid_dir))
print("Testing:", count_images(test_dir))


Image counts:
Training: {'CaS': 480, 'CoS': 450, 'Gum': 360, 'MC': 540, 'OC': 324, 'OLP': 540, 'OT': 393}
Validation: {'CaS': 160, 'CoS': 149, 'Gum': 120, 'MC': 180, 'OC': 108, 'OLP': 180, 'OT': 131}
Testing: {'CaS': 160, 'CoS': 149, 'Gum': 120, 'MC': 180, 'OC': 108, 'OLP': 180, 'OT': 131}


In [5]:
# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=True
)

validation_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASS_NAMES,
    shuffle=False
)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [6]:
# Create base model
base_model = EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Create model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, 
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(len(CLASS_NAMES), activation='softmax')
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# Train the model
initial_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=40,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=8,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5
        )
    ]
)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


97/97 ━━━━━━━━━━━━━━━━━━━━ 225s 1s/step - accuracy: 0.5226 - loss: 3.8394 - precision: 0.5943 - recall: 0.4568 - val_accuracy: 0.1449 - val_loss: 4.3306 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 37s 353ms/step - accuracy: 0.8854 - loss: 2.0729 - precision: 0.9046 - recall: 0.8659 - val_accuracy: 0.1449 - val_loss: 3.4849 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 37s 357ms/step - accuracy: 0.9237 - loss: 1.3612 - precision: 0.9306 - recall: 0.9109 - val_accuracy: 0.1449 - val_loss: 3.1824 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 36s 348ms/step - accuracy: 0.9451 - loss: 0.8416 - precision: 0.9540 - recall: 0.9359 - val_accuracy: 0.2160 - val_loss: 2.9149 - val_precision: 0.2044 - val_recall: 0.0632 - learning_rate: 0.0010
Epoch 5/40
97/97 ━━━━━━━━━━━━━━━━━━━━ 37s 358ms/s

In [8]:
# Prepare model for fine-tuning
base_model = model.layers[0]
base_model.trainable = True

# Freeze all layers except the last 15
for layer in base_model.layers[:-15]:
    layer.trainable = False

# Recompile model with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=6,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=4,
            min_lr=1e-7
        )
    ]
)

Epoch 1/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 92s 596ms/step - accuracy: 0.9963 - loss: 0.0207 - precision: 0.9963 - recall: 0.9963 - val_accuracy: 0.9971 - val_loss: 0.0138 - val_precision: 0.9971 - val_recall: 0.9971 - learning_rate: 1.0000e-05
Epoch 2/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 333ms/step - accuracy: 0.9988 - loss: 0.0161 - precision: 0.9988 - recall: 0.9986 - val_accuracy: 0.9971 - val_loss: 0.0132 - val_precision: 0.9971 - val_recall: 0.9971 - learning_rate: 1.0000e-05
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 338ms/step - accuracy: 0.9992 - loss: 0.0133 - precision: 0.9992 - recall: 0.9991 - val_accuracy: 0.9971 - val_loss: 0.0126 - val_precision: 0.9971 - val_recall: 0.9971 - learning_rate: 1.0000e-05
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 34s 329ms/step - accuracy: 0.9981 - loss: 0.0158 - precision: 0.9981 - recall: 0.9981 - val_accuracy: 0.9990 - val_loss: 0.0121 - val_precision: 0.9990 - val_recall: 0.9990 - learning_rate: 1.0000e-05
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 33

In [9]:
# Evaluate model
test_results = model.evaluate(test_generator)
print("\nTest Results:")
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Accuracy: {test_results[1]:.4f}")
print(f"Test Precision: {test_results[2]:.4f}")
print(f"Test Recall: {test_results[3]:.4f}")

# Save model
model.save('teeth_classification_model.h5')

33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 134ms/step - accuracy: 0.9984 - loss: 0.0081 - precision: 0.9984 - recall: 0.9984

Test Results:
Test Loss: 0.0112
Test Accuracy: 0.9971
Test Precision: 0.9971
Test Recall: 0.9971
